In [365]:
import pandas as pd
from datetime import datetime
from datetime import timedelta 

In [366]:
def make_oil(df):
  data = pd.DataFrame(columns=["url", "title", "date", "id", "tags", "description"])
  for i in range(len(df)):
    if "Generic 1st 'CL' Future" in df['tags'][i] or "oil" in str(df['title'][i]).lower() or 'oil' in str(df['description'][i]).lower():
      try: date = datetime.strptime(df['date'][i], '%Y-%m-%d').date()
      except: continue
      data = data.append({"url": df['url'][i], "title": df['title'][i], "date": date, "id": df['id'][i], "tags": df['tags'][i], "description": df['description'][i]}, ignore_index=True)
  return data

In [367]:
def find_price(date, prices):
  for j in range(len(prices)):
      if (str(date) == str(prices['date'][j])):
        return prices['price'][j]
  return False

In [368]:
def find_drill(date, drills):
  for j in range(len(drills)):
      if (str(date) == str(drills['date'][j])):
        return [drills['fact'][j], drills['last'][j]]
  return False

In [369]:
def make_prices(prices, oil):
  out = pd.DataFrame(columns=["url", "title", "date", "id", "tags", "description", "price", "next_day_price", "next_week_price", "next_day", "next_week"])
  for i in range(len(oil)):
    date = oil['date'][i]
    date2 = date
    date3 = date + timedelta(days=1)
    date4 = date + timedelta(days=7)
    while True:
      price = find_price(date2, prices)
      next_day_price = find_price(date3, prices)
      next_week_price = find_price(date4, prices)
      if date3 > datetime.now().date(): next_day_price = 1
      if date4 > datetime.now().date(): next_week_price = 1
      if price and next_day_price and next_week_price: break
      if price == False: date2 -= timedelta(days=1)
      if next_day_price == False: date3 += timedelta(days=1)
      if next_week_price == False: date4 += timedelta(days=1)
    if float(price) > float(next_day_price): next_day = 0
    else: next_day = 1
    if float(price) > float(next_week_price): next_week = 0
    else: next_week = 1
    out = out.append({
        "url": oil['url'][i],
        "title": oil['title'][i],
        "date": date,
        "id": oil['id'][i],
        "tags": oil['tags'][i],
        "description": oil['description'][i],
        "price": price,
        "next_day_price": next_day_price,
        "next_week_price": next_week_price,
        "next_day": next_day,
        "next_week": next_week
        }, ignore_index=True)
  return out

In [370]:
def make_drills(drills, oil):
  out2 = pd.DataFrame(columns=["url", "title", "date", "id", "tags", "description", "price", "next_day_price", "next_week_price", "next_day", "next_week", "drills_fact", "drills_last"])
  for i in range(len(oil)):
    date = oil['date'][i]
    date2 = date
    while True:
      drill = find_drill(date2, drills)
      if drill: break
      if drill == False: date2 -= timedelta(days=1)
    drills_fact = drill[0]
    drills_last = drill[1]
    out2 = out2.append({
        "url": oil['url'][i],
        "title": oil['title'][i],
        "date": str(date),
        "id": oil['id'][i],
        "tags": oil['tags'][i],
        "description": oil['description'][i],
        "price": oil['price'][i],
        "next_day_price": oil['next_day_price'][i],
        "next_week_price": oil['next_week_price'][i],
        "next_day": oil['next_day'][i],
        "next_week": oil['next_week'][i],
        "drills_fact": drills_fact,
        "drills_last": drills_last
        }, ignore_index=True)
  return out2

In [371]:
df = pd.read_csv('data - копия.csv')
oil1 = make_oil(df)
# print(oil1)

                                                    url  \
0     https://www.bloomberg.com/news/articles/2020-0...   
1     https://www.bloomberg.com/news/articles/2020-0...   
2     https://www.bloomberg.com/news/articles/2020-0...   
3     https://www.bloomberg.com/news/articles/2020-0...   
4     https://www.bloomberg.com/news/articles/2020-0...   
...                                                 ...   
2188  https://www.bloomberg.com/news/articles/2020-0...   
2189  https://www.bloomberg.com/news/articles/2020-0...   
2190  https://www.bloomberg.com/news/articles/2020-0...   
2191  https://www.bloomberg.com/news/articles/2020-0...   
2192  https://www.bloomberg.com/news/articles/2020-0...   

                                                  title        date  \
0     Why We Still Can’t Stop Oil Spills From Damagi...  2020-09-10   
1     One of the World’s Richest Petrostates Is Runn...  2020-09-02   
2     Apollo Group Invests $2.7 Billion in UAE Oil F...  2020-09-02   
3     H

In [372]:
prices = pd.read_csv('prices.csv')
oil2 = make_prices(prices, oil1)
# print(oil2)

In [373]:
drills = pd.read_csv('convertcsv.csv')
oil3 = make_drills(drills, oil2)
# print(oil3)

In [374]:
#accidents = pd.read_csv('accidents.csv')

In [375]:
#print(accidents.keys())

In [376]:
# print(accidents['Net Loss (Barrels)'])
# print(accidents['All Fatalities'])
# print(accidents['All Costs'])

In [377]:
oil3.to_csv('result.csv', index=False)

In [378]:
# df = pd.read_csv('data - копия.csv')
# prices = pd.read_csv('prices.csv')
# drills = pd.read_csv('convertcsv.csv')
# oil = make_oil(df)
# oil = make_prices(prices, oil)
# oil = make_drills(drills, oil)
# oil.to_csv('test.csv', index=False)